## EDA on Payment Amount
- This EDA  is being done by Lori & Teresa.

Covered in this notebook:
1. First approach: Teresa and I were concerned that outliers have been 'buried' in the combined/grouped dfs. We went back to working with 2015 data (me) and 2016 data (Teresa) to see if we can find outliers that way.
2. That approach didn't yeild results we could use, so we then started to look at the best way to get to the business question were were attempting to answer: Find the change between 2015 and 2017 in average allowed amount, based on three criteria (payment_type = Doctor Only, Facility Only, or Doctor & Facility; NPI and HCPCS code)
- We attempted to do that in Python, but ran into problems with the data (the '201x' column name was an integer, not a string!).  After resolving that, though, we still couldn't find the difference between 2015 adn 2017.
- After consulting with the rest of the team (who had run into similar problems), the decision was made to export the pivoted df to .csv and continue the manipulations in Tableau - for both the allowed amount (that our sub-team is working on - Deigo, Teresa and me); and the other team who's working on the utilization change (Ari, Nicole and Jack).

#### Reminder of questions our sub-team is answering: 
    - Goals 2: Which procedures had the largest change in Average payment?

In [ ]:
import pandas as pd
import pickle
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Reading in 2015 pickle file (clean; doesn't have the irrelevant row)

df_payments_2015 = pd.read_pickle('../data/pickled_files/payments_2015.pkl')
print(df_payments_2015.shape)
df_payments_2015.head()

In [ ]:
# Reading in 2016 pickle file (clean; doesn't have the irrelevant row)

df_payments_2016 = pd.read_pickle('../data/pickled_files/payments_2016.pkl')
print(df_payments_2016.shape)
df_payments_2016.head()

In [ ]:
# Reading in 2017 pickle file (clean; doesn't have the irrelevant row)

df_payments_2017 = pd.read_pickle('../data/pickled_files/payments_2017.pkl')
print(df_payments_2017.shape)
df_payments_2017.head()

In [ ]:
df_payments_2015.tail()

### Adding new column with payment-type  (Doctor Only, Facility Only, or Doctor & Facility)
(Code from Diego)

In [ ]:
# Step 1 to add new column with payment type (Code from Deigo)

conditions = [
    (df_payments_2015.place_of_service == 'O'),
    (df_payments_2015.entity_type_of_the_provider == 'I') & (df_payments_2015.place_of_service == 'F'),
    (df_payments_2015.entity_type_of_the_provider == 'O') & (df_payments_2015.place_of_service == 'F')
]

choices = ['Doctor & Facility', 'Doctor Only', 'Facility Only']

In [ ]:
# Step 2 to add new column with payment type (Code from Deigo)

df_payments_2015['payment_type'] = np.select(conditions, choices, default = 'unknown')

In [ ]:
# Confirmed that there are no "unknown" values.  THERE ARE UNKNOWNS. RESEARCHING FURTHER

df_payments_2015['payment_type'].unique()

In [ ]:
df_payments_2015.columns

In [ ]:
df_pmts_subset_2015 = df_payments_2015[['year',
                                       'payment_type', 
                                       'hcpcs_code', 
                                       'average_medicare_allowed_amount'
                                      ]]

In [ ]:
df_pmts_subset_2015.head()

In [ ]:
type(df_pmts_subset_2015)

In [ ]:
%%time
# Takes a long time to run
# This shows that there are a few outliers, but not in a usable way.

print(df_pmts_subset_2015.plot())

### Clarifying what is aggregated up into combined df

In [ ]:
# To read in the combined df with all years, that was created in step03 notebook

df_payments_combined = pd.read_pickle('../data/pickled_files/payments_combined.pkl')
print(df_payments_combined.shape)
df_payments_combined.head()

In [ ]:
df_payments_combined.columns

In [ ]:
# To pivot the df_payments_combined so that each year is column header (not a value under "year")
# Code is from Diego

pivot_index = ['national_provider_identifier',
               'entity_type_of_the_provider', 
               'place_of_service',
               'payment_type',
               'provider_type',
               'hcpcs_code',
               'hcpcs_description',
               'zip_code_of_the_provider', 
               'state_code_of_the_provider']

pivot_cols = ['year']

In [ ]:
%%time
df_pmt_pvt = df_payments_combined.pivot_table(index = pivot_index, 
                                              columns = pivot_cols, 
                                              values = 'average_medicare_allowed_amount', 
                                              aggfunc=np.mean)
df_pmt_pvt = df_pmt_pvt.reset_index()

In [ ]:
# Checking the shape and head of the combined df. Looks correct. 20793075 rows, 12 columns.

print(df_pmt_pvt.shape)
df_pmt_pvt.head().reset_index()

In [ ]:
# Confirming the type, to ensure it's a df.

type(df_pmt_pvt)

In [ ]:
df_pmt_pvt.to_pickle('../data/pickled_files/payments_pivoted')

In [ ]:
df_pmt_pvt.columns

In [ ]:
# Creating df to get median based on pmt type and hcpcs code, by using "groupby" on pmt_type and hcpcs_code.
# This didn't give the desired result - too many null values in year columns. 
# Back to the drawing board.

df_pmt_pvt_group = df_pmt_pvt.groupby(['payment_type',
                                          'hcpcs_code'])

df_pmt_pvt_group.head()

### Next attempts: 
**FIRST: Create new column for type of payment in 2015**
- Creating the new column for payment_type (Doctor & Facility, etc.) in the 2015 df, so we can use that to do groupby.
- Successful in creating the column, but still unable to find what we needed from this.

**SECOND: Try to create combined table in a different way:**
- The next thing we did was add this column also to 2016 and 2017, to then MERGE them (instead of concatenating, then pivoting) to see if that would help with the calculations.  It still didn't work. 

In [ ]:
# Adding payment_type column to 2015

conditions = [
    (df_payments_2015.place_of_service == 'O'),
    (df_payments_2015.entity_type_of_the_provider == 'I') & (df_payments_2015.place_of_service == 'F'),
    (df_payments_2015.entity_type_of_the_provider == 'O') & (df_payments_2015.place_of_service == 'F')]

choices = ['Doctor & Facility', 'Doctor Only', 'Facility Only']

In [ ]:
# Adding payment_type column to 2015

%%time
df_payments_2015['payment_type'] = np.select(conditions, choices, default = 'unknown')

In [ ]:
# Adding payment_type column to 2015

df_payments_2015.head(2)

In [ ]:
# Adding payment_type column to 2016

conditions = [
    (df_payments_2016.place_of_service == 'O'),
    (df_payments_2016.entity_type_of_the_provider == 'I') & (df_payments_2016.place_of_service == 'F'),
    (df_payments_2016.entity_type_of_the_provider == 'O') & (df_payments_2016.place_of_service == 'F')]

choices = ['Doctor & Facility', 'Doctor Only', 'Facility Only']

In [ ]:
# Adding payment_type column to 2016

%%time
df_payments_2016['payment_type'] = np.select(conditions, choices, default = 'unknown')
df_payments_2016.head(2)

In [ ]:
# Adding payment_type column to 2017

conditions = [
    (df_payments_2017.place_of_service == 'O'),
    (df_payments_2017.entity_type_of_the_provider == 'I') & (df_payments_2017.place_of_service == 'F'),
    (df_payments_2017.entity_type_of_the_provider == 'O') & (df_payments_2017.place_of_service == 'F')]

choices = ['Doctor & Facility', 'Doctor Only', 'Facility Only']

In [ ]:
# Adding payment_type column to 2017

%%time
df_payments_2017['payment_type'] = np.select(conditions, choices, default = 'unknown')
df_payments_2017.head(2)

In [ ]:
df_payments_2016.columns

In [ ]:
# Dropping columns we don't need.
# This code works, but takes a long time, so it's been commented out to prevent running when not needed.


'''
df_payments_2016_subset = df_payments_2016.drop(columns = ['last_name_organization_name_of_the_provider',
                                                           'entity_type_of_the_provider',
                                                           'city_of_the_provider',
                                                           'zip_code_of_the_provider',
                                                           'state_code_of_the_provider',
                                                           'provider_type','hcpcs_description',
                                                           'number_of_services',
                                                           'number_of_medicare_beneficiaries',
                                                           'number_of_distinct_medicare_beneficiary_per_day_services'])
#                                                           'year'])
'''

In [ ]:
# Merge (outer) on 2015 and 2016. Our plan was to first merge these two, then merge 2017 with the new df.
# But the merge didn't work well. Returned lots of duplicated columns with column_name_x and column_name_y.

df_pmt_combined_merge = pd.merge(df_payments_2015,
                                df_payments_2016,
                                how='outer',
                                on=['national_provider_identifier',
                                    'hcpcs_code',
                                    'payment_type'
                                   ])

In [ ]:
df_pmt_combined_merge.head()


##  Work on Friday 6/5/2020 - we intended to do the following. However, with live-coding session, we discovered that our original data (the combined df created a few days ago df_avg_pmt contained many duplicate rows. 
- We would need to go back and fix that before proceeding.  
- The other sub-team had managed to solve the original issue in Tableau that had caused us to split off and try this method in the first place, so it wasn't a good use of time to try and fix this.  
- DataFrames we would have created, given time and need:  
**df_pmt_melt_amt** (average) 2-year change in ALLOWED AMOUNT, by AMOUNT  
**df_pmt_melt_pct** (average) 2-year change in ALLOWED AMOUNT, by PERCENT  
**df_util_melt_number** (sum) 2-year change in BEN/DAY SERVICES, by NUMBER  
**df_util_melt_pct** (sum) 2-year change in BEN/DAY SERVICES, by PERCENT  

